In [74]:
import pandas as pd
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [141]:
chrome_options = Options()
#chrome_options.add_argument("--remote-debugging-port=9222")
#chrome_options.add_argument('--headless')
browser = webdriver.Chrome(options=chrome_options)
# first get gare de lyon directions page
browser.get('https://www.google.com/maps/dir//Paris+Gare+de+Lyon,+Place+Louis-Armand,+75571+Paris,+France/@48.8443038,2.372194,17z/data=!4m8!4m7!1m0!1m5!1m1!1s0x47e6721b2000bf4b:0x3ca1c23108bb64!2m2!1d2.3743773!2d48.8443038?hl=en')

location = browser.find_element_by_css_selector("[aria-label='Choose starting point, or click on the map...']")
ActionChains(browser)\
    .move_to_element(location).click()\
    .send_keys('saint denis')\
    .send_keys(Keys.ENTER)\
    .perform();

wait = WebDriverWait(browser, 20)
departure = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[aria-label='Departure options']")))
departure.click()
#departure = browser.find_element_by_css_selector("[aria-label='Departure options']")
#ActionChains(browser)\
#    .move_to_element(departure).click().perform();
depart = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "goog-menuitem")))

depart = browser.find_elements_by_class_name("goog-menuitem")[1]
ActionChains(browser)\
    .move_to_element(depart).click().perform();

timer = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "time-input")))
timer = browser.find_element_by_class_name("time-input")
timer.clear()
ActionChains(browser)\
    .move_to_element(timer).click()\
    .send_keys('8:24 AM').perform();

date = browser.find_element_by_class_name("date-input")
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + datetime.timedelta(days_ahead)

d = datetime.date.today()
next_monday = next_weekday(d, 0).strftime("%d %b") # 0 = Monday, 1=Tuesday, 2=Wednesday...
next_monday = next_monday[1:] if next_monday[0]=='0' else next_monday

ActionChains(browser)\
    .move_to_element(date).click().perform();
monday = browser.find_element_by_css_selector("[aria-label='"+next_monday+"']")
ActionChains(browser)\
    .move_to_element(monday).click().perform();


transit = browser.find_element_by_css_selector("[aria-label='Transit']")
ActionChains(browser)\
    .move_to_element(transit).click().perform();

trips = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "section-directions-trip")))
trips = browser.find_elements_by_class_name("section-directions-trip")

ActionChains(browser)\
    .move_to_element(trips[-1]).click().perform();
time.sleep(1)    
for trip in trips:
    index = trips.index(trip)
    print('Trip ',index+1," : " )
    ActionChains(browser)\
    .move_to_element(trip).click().perform();

    trips_title = trip.find_element_by_class_name("section-directions-trip-title")
    print(trips_title.text)
    
    trips_duration = trip.find_element_by_class_name("section-directions-trip-duration")
    print(trips_duration.text)
    trips_summary = trip.find_element_by_class_name("section-directions-trip-renderable-summary")
    print(trips_summary.text)
    trips_info = trip.find_element_by_class_name("section-directions-trip-summary")
    print(trips_info.text)
    
#walking = browser.find_element_by_css_selector("[aria-label='Walking']")
#ActionChains(browser)\
#    .move_to_element(walking).click().perform();

#cycling = browser.find_element_by_css_selector("[aria-label='Cycling']")
#ActionChains(browser)\
#    .move_to_element(cycling).click().perform();

arkose = ['arkose nation','arkose pantin','arkose montreuil','arkose didot','arkose pont de sèvres','arkose massy']
timer.clear()
ActionChains(browser)\
    .move_to_element(timer).click()\
    .send_keys('7:40 PM').perform();

destination = browser.find_elements_by_class_name("tactile-searchbox-input")[1]
destination.clear()

for gym in arkose:
    ActionChains(browser)\
        .move_to_element(destination).click()\
        .send_keys(gym)\
        .send_keys(Keys.ENTER)\
        .perform();

    trips = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "section-directions-trip")))
    #show only first trip for gym
    trip_duration = browser.find_element_by_class_name("section-directions-trip-duration")
    print(gym," : ",trip_duration.text)


Trip  1  : 
8:26 AM—8:45 AM
19 min
RER D
8:26 AM from Saint-Denis
Trip  2  : 
8:32 AM—8:49 AM
17 min
RER D
8:32 AM from Saint-Denis
Trip  3  : 
8:36 AM—8:53 AM
17 min
RER D
8:36 AM from Saint-Denis
Trip  4  : 
8:41 AM—9:00 AM
19 min
RER D
8:41 AM from Saint-Denis
arkose nation  :  28 min
arkose pantin  :  55 min
arkose montreuil  :  40 min
arkose didot  :  55 min
arkose pont de sèvres  :  53 min
arkose massy  :  1 hr
